### Handlabeling Negative-Positive Evaluator


Machine Learning Approach:



In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import pandas as pd
import numpy as np
import re
import math
import statistics
from hatesonar import Sonar

In [2]:
def mostCommon(li):
    st = set(li)
    mx = -1
    for each in st:
        temp = li.count(each)
        if mx < temp:
            mx = temp 
            h = each 
    return h

In [3]:
#reading the labeling csv
data = pd.read_csv('../Data/dataset_handlabeling.csv', sep=',') 

# analyze
analyzer = SentimentIntensityAnalyzer() 
# load learnables
voters = list(data.columns.values)
voters.remove('AD_TEXT')

for index, row in data.iterrows():
    decisions = []
    for voter in voters:
        decisions.append(row[voter].lower())
    convergenceValue = mostCommon(decisions)
    data.loc[index, 'vote'] = convergenceValue
    

In [15]:

# 10 random shuffel for 10 random samples
averageAccuracy = []
averageRecall = []

for x in range (20):
    selectedData = data.sample(12)
    # dictionary analyzer
    selectedData['pos'] = np.nan
    selectedData['neg'] = np.nan
    selectedData['neu'] = np.nan
    selectedData['compound'] = np.nan
    selectedData['flag'] = np.nan
    
    matchCount = 0
    totalCount = 0
    for index, row in selectedData.iterrows():
        # skip empty texts
        if pd.isnull(row['AD_TEXT']):
            continue

        # calculate the senti-index for each advert text
        tempRespond = analyzer.polarity_scores(row['AD_TEXT'])    

        selectedData.loc[index, 'pos'] = tempRespond['pos']
        selectedData.loc[index, 'neg'] = tempRespond['neg']
        selectedData.loc[index, 'neu'] = tempRespond['neu']
        selectedData.loc[index, 'compound'] = tempRespond['compound']

        # make the flags
        if 0.5 <= tempRespond['compound']  and tempRespond['compound'] <= 1:
            selectedData.loc[index, 'flag'] = 'positive'
        elif -0.5 < tempRespond['compound']  and tempRespond['compound'] < 0.5:
            selectedData.loc[index, 'flag'] = 'neutral'
        else:
            selectedData.loc[index, 'flag'] = 'negative'
        
        # accuracy ignoring neutral votes
        if selectedData.loc[index, 'flag'] == 'neutral': continue
        if selectedData.loc[index, 'flag'] == row['vote']:
            matchCount = matchCount + 1
        totalCount = totalCount + 1
    
    accuracy = matchCount / totalCount * 100
    print( 'Accuracy Trail', accuracy, ' %' )
    averageAccuracy.append(accuracy)



Accuracy Trail 85.71428571428571  %
Accuracy Trail 85.71428571428571  %
Accuracy Trail 50.0  %
Accuracy Trail 83.33333333333334  %
Accuracy Trail 71.42857142857143  %
Accuracy Trail 75.0  %
Accuracy Trail 62.5  %
Accuracy Trail 75.0  %
Accuracy Trail 66.66666666666666  %
Accuracy Trail 71.42857142857143  %
Accuracy Trail 71.42857142857143  %
Accuracy Trail 62.5  %
Accuracy Trail 100.0  %
Accuracy Trail 83.33333333333334  %
Accuracy Trail 75.0  %
Accuracy Trail 85.71428571428571  %
Accuracy Trail 77.77777777777779  %
Accuracy Trail 83.33333333333334  %
Accuracy Trail 50.0  %
Accuracy Trail 85.71428571428571  %


In [16]:
print ('Average Accuracy',  statistics.mean(averageAccuracy) , ' %')

Average Accuracy 75.07936507936508  %
